In [1]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import os
import numpy as np
import pickle
from keras.models import Model, Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers import Bidirectional
from keras.layers import LSTM
from keras.layers import TimeDistributed
from numpy import zeros

Using TensorFlow backend.


In [0]:
class Language:
  def __init__(self):
    token_list = ["This", "is", "my", "sentence"]
    self.max_length = self.get_max_length(token_list)
    sentences= [' '.join(tokens) for tokens in token_list]
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(sentences)
    self.tokenizer = tokenizer
    self.vocab =  len(tokenizer.word_index) + 1
    self.word2vec = self.get_word2vec("/content/glove.twitter.27B.100d.txt")
    self.embedding_matrix = self.get_embedding_matrix()
  
  def get_max_length(self, token_list):
    max_len = 0
    for idx, tokens in enumerate(token_list):
      if len(tokens) > max_len:
        max_len = len(tokens)
    return max_len
  
  def dump(self, data, filepath):
    with open(filepath, "wb") as f:
      pickle.dump(data, f)


  def get_word2vec(self, file_path):
    file_w2v = "/content/w2v_100.pkl"
    exists = os.path.isfile(file_w2v)
    if exists:                                      # load a prexisting pickel module
      word2vec = pickle.load(open(file_w2v,"rb"))
      return word2vec
    else:
      file = open(file_path, "r",encoding="utf8")
      if (file):
        word2vec = dict()
        split = file.read().splitlines()
        for line in split:
          key = line.split(' ', 1)[0]  # the first word is the key
          value = np.array([float(val) for val in line.split(' ')[1:]])
          word2vec[key] = value
        self.dump(word2vec, os.path.join("w2v.pkl"))
        print("dumped the w2v file")

  def get_embedding_matrix(self):
    embedding_matrix = zeros((self.vocab,25))
    for word, i in self.tokenizer.word_index.items():
      embedding_vector = self.word2vec.get(word)
      if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
      return embedding_matrix

  def get_encoded_data(self,tokens):
    sentences = [' '.join(tokens) for tokens in tokens]
		# integer encode the documents
    encoded_docs = self.tokenizer.texts_to_sequences(sentences)
    # pad documents to a max length of 4 words
    lng_data = pad_sequences(encoded_docs, maxlen=self.max_length, padding='post')
    return lng_data

In [4]:
language = Language()
tokens = ["This", "is", "my", "sentence"]
print(language.get_encoded_data(tokens))

FileNotFoundError: ignored